In [3]:
%load_ext autoreload
%autoreload 2
%env CUDA_VISIBLE_DEVICES = 0


from batchflow import *
# from batchflow.named_expr
from batchflow.opensets import *
from batchflow.models.torch import *
from batchflow.models.torch.layers import *
from load_segment_dataset import Segmentation_dataset
from torch.optim import Adam
import torch
from model_learning import Train
import torch.nn.functional as F

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: CUDA_VISIBLE_DEVICES=0


In [4]:
mnist = MNIST()

In [6]:
mnist.p

In [104]:
ppl = Pipeline() << Dataset(100)
ppl = mnist.p

In [105]:
def foo(batch, iteration):
    print(f'{id(batch)} ::: indices={batch.indices} ::: {iteration=}')
    
    images, labels = next(dataloader)
    # images = [dataloader[i][0] for i in batch.indices]
    batch.images = images
    

ppl += (Pipeline()
        # .foo(B(), iteration=I())
        .to_array(src=B.images, dst='images', channels='first', dtype='float32')
        # .multiply(src='images', dst='images', multiplier=1/255., preserve_type=False)
)

In [100]:
batch = ppl.next_batch(batch_size=4, shuffle=True)

140350429343504 ::: indices=[73 15 60 37] ::: iteration=1


In [43]:
batch.images, batch.labels

(array([<PIL.Image.Image image mode=L size=28x28 at 0x7FA5E2C41280>,
        <PIL.Image.Image image mode=L size=28x28 at 0x7FA5E3B5B8B0>,
        <PIL.Image.Image image mode=L size=28x28 at 0x7FA5E30B6CA0>,
        <PIL.Image.Image image mode=L size=28x28 at 0x7FA5E32C10D0>],
       dtype=object),
 array([9, 7, 0, 5], dtype=uint8))

In [30]:
batch.index.index

array([6, 8, 7, 4])

In [108]:
ppl.run(n_iters=1000, batch_size=10, notifier='t')

100%|██████████████████████████████████████| 1000/1000 [00:01<00:00, 698.12it/s]


In [46]:
for batch in ppl.gen_batch(n_iters=10, batch_size=4):
    #some actions

140350842931136 ::: indices=[0 1 2 3]
140350842929696 ::: indices=[4 5 6 7]
140350448185200 ::: indices=[ 8  9 10 11]
140350448181600 ::: indices=[12 13 14 15]
140350842930032 ::: indices=[16 17 18 19]
140350448181456 ::: indices=[20 21 22 23]
140350842931136 ::: indices=[24 25 26 27]
140350448181600 ::: indices=[28 29 30 31]
140350842929648 ::: indices=[32 33 34 35]
140350448181456 ::: indices=[36 37 38 39]


In [11]:
ppl.dataset

In [ ]:
for images, labels in dataloader:
    ###
    batchflow_model.train(...)

In [ ]:
DEVICE = 'cuda:0'

In [ ]:
config = { 'inputs_shapes': (3, 256, 256),     
    'order': ['encoder', 'embedding', 'decoder', 'head'], 

    'encoder': {              
        'type': 'encoder',                                                   
        'num_stages': 4,  
        'order':['blocks', 'skip', 'downsample'],   
        'blocks':{'layout':'cna cna', 'channels':[64, 128, 256, 512]},   
        'downsample': {'layout': 'p', 'pool_size': 2, 'pool_stride': 2},
    },
    'embedding': {   
        'layout': 'cna cna',
        'channels': [1024, 1024], 
        'input_type': 'list',  
        'output_type': 'list',
        'input_index': -1,
    },
    'decoder': {
        'order':['upsample', 'combine', 'blocks'],
        'type': 'decoder',           
        'num_stages': 4,
        'blocks':{'layout':'cna cna', 'channels':['same // 2', 'same']},                                                     
        'upsample': {'layout': 't', 'kernel_size': 2, 'channels':'same // 2'},  
        'combine': {'op': 'concat'},   },             
        
    'head': {                     
        'layout': 'c', 
        'channels': 151,
        'kernel_size':1},
    'loss': 'ce',
    'device': DEVICE
    
}

In [ ]:
def get_segmentation_config(model_class, config):

    if 'inputs_shapes' in config and isinstance(config['inputs_shapes'], list):
        inputs = [B.images for item in config['inputs_shapes']]
    else:
        inputs = B.images ## B
    
    pipeline_config = {
        'model': model_class,
        'model_config': {**config},
        'inputs': inputs,
        'targets': B.images,

        'gather': {'metrics_class' : 'segmentation',  # gather
                   'fmt' : 'proba',                   # fmt: proba
                   
                   'axis' : None,
                   'targets' : B.images},
        'evaluate': 'jaccard',
    }
    return pipeline_config

In [ ]:
def get_pipeline(pipeline_config):
    """ Pipeline config must contain 'model', 'model_config', 'feed_dict' keys. """
    pipeline = (Pipeline(config=pipeline_config)
                .init_variable('loss_history', [])
                .to_array(channels='first', dtype='float32')
                .multiply(multiplier=1/255., preserve_type=False)
                .init_model(name='MODEL', model_class=C('model'), config=C('model_config'))
                .train_model('MODEL',
                             inputs=B.images,
                             targets=B.masks,
                             outputs='loss',
                             save_to=V('loss_history', mode='a'))
                )
    return pipeline

In [ ]:
def run(model_class, config, batch_size=BATCH_SIZE, n_iters=N_ITERS, **kwargs):
    pipeline_config = get_segmentation_config(model_class, config)

    train_pipeline = get_pipeline(pipeline_config) << mnist.train   #
    _ = train_pipeline.run(batch_size, n_iters=n_iters,
                           bar={'bar': BAR, 'monitors': 'loss_history'},
                           **kwargs)
    
    print(f'Segmentation Unet is done! Number of parameters in the model: {train_pipeline.model.num_parameters:,}')
    return train_pipeline

In [ ]:
ppl = run(model_class=TorchModel, config=config, n_iters=2, batch_size=4)